# Bayes Classifier. Maximum Likelihood for a multivariate Gaussian density.

## Preface

Begin, if necessary, by recalling the course notes on the [Bayes classifier](https://studium.umontreal.ca/pluginfile.php/4027948/mod_resource/content/4/7_bayes_classifier-en.pdf) and the principle of [maximum likelihood](https://studium.umontreal.ca/pluginfile.php/4003963/mod_resource/content/4/5_gaussian_distribution_en.pdf).

## High level description

Today we are going to build a **multi-class Bayes classifier**. This means that instead of modeling $ p (\mbox{class} | \mbox{example}) $ (or $ p (y | x) $), we will instead use the Bayes equation

$$ p (\mbox{class} | \mbox{example}) = \frac{p (\mbox{example} | \mbox{class}) p (\mbox {class})} {\sum_{c'= 1}^{m} p_\mbox{c'}(x) P_\mbox{c'}} $$

and model the different pieces. In fact, we just need to model the numerator since the denominator is a normalization constant. In addition, $ P_\mbox{c '} = n_c / n $

The term $ p (\mbox{class}) $ represents the prior probability of a class, that is, our a priori belief - before we have seen a particular example - about the probability that an unknown example belongs to this class). We will represent this belief a priori for a class by the frequency of the latter in the training data: $ \frac{n_c}{n} $ where $ n_c $ = number of examples of the class $ c $, and $ n $ = number of training examples.

We will use **multivariate Gaussian densities** to model the different $ p (\mbox{example} | \mbox{class}) $. This means that for each class, we will assume that the "true" distribution $ p (\mbox{example} | \mbox{class}) $ has the form of a multivariate Gaussian for which we will try to learn the parameters $ \mu $ and $ \Sigma $. In practice, we will limit ourselves today to a particular case of this distribution: the one where we assume that the covariance matrix $ \Sigma $ of each Gaussian is diagonal and that each element of this diagonal is the same, i.e. sigma_sq (<=> "sigma square" <=> $ \sigma^2 $ <=> the variance). Thus we have a single parameter to control the expression of the covariance. It's easier (for us and for the computer) to calculate, but it also means that our model is less powerful.

So we have a very simple parametric model. The parameters are the average $ \mu $ (a vector of the same dimension as the dimension of the system input) and the variance $ \sigma^2 $ (a single scalar in our simple model, which will multiply the identity matrix). Learning in this model will be done today by applying the **maximum likelihood principle**. For each class, we will find the values of the parameters that maximize the log-likelihood of the training data from this class:

$$ \log \prod_i^n p(X = x_i) $$

For an insotropic Gaussian of dimension $d$, the maximum likelihood estimators of $\mu$ and $\sigma^2$ are given by: 

$$\mu_{ML} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

$$\sigma_{ML}^2 = \frac{1}{nd} \sum_{i=1}^{n} (x_i-\mu_{MV})^T(x_i-\mu_{MV})$$

Having found the parameters that maximize the likelihood for each class, we can calculate each $ p (\mbox{example} | \mbox{class}) $. It is now sufficient to apply the Bayes rule in order to classify a new example. More precisely, we want to choose, for an example, the class that maximizes $ p(\mbox{example} | \mbox{class}) p(\mbox{class}) $ or, equivalently, $ \log (p (\mbox{example } | \mbox{class} ) p(\mbox{class})) $.

## Code to be completed

Load the file `utilities.py` into the folder where your notebook files are located. It contains the useful functions you saw in the last class. You can thus use them without cluttering your notebook.

For the `gauss_ml` class:
 
  - calculate sigma_sq ($ \sigma^2 $), the variance in `gauss_ml.train`
  - calculate the value of the Gaussian density function in `gauss_ml.compute_predictions`

In [1]:
#%pylab inline
import numpy as np
import utilities

In [46]:
class gauss_ml:
    def __init__(self,n_dims,cov_type="isotropic"):
        self.n_dims = n_dims
        self.mu = np.zeros((1,n_dims))
        self.cov_type = cov_type
        
        if cov_type=="isotropic":
            self.sigma_sq = 1.0
        if cov_type=="full":
            pass

	# For a training set, the function should compute the ML estimator of the mean and the covariance matrix
    def train(self, train_data):
        self.mu = np.mean(train_data,axis=0)
        if self.cov_type == "isotropic":
            self.sigma_sq = ((1./ train_data.shape[0] * self.n_dims) * 
                             np.sum(np.matmul(np.transpose(train_data - self.mu), train_data - self.mu)))
        if self.cov_type == "full":
            pass

	# Returns a vector of size nb. of test ex. containing the log
	# probabilities of each test example under the model.	
    def compute_predictions(self, test_data):
        
        log_prob = -np.ones((test_data.shape[0],1))
        print(self.mu)
        if self.cov_type == "isotropic":
            # the following line calculates log(normalization constant)
            c = -self.n_dims * np.log(2*np.pi)/2 - self.n_dims*np.log(np.sqrt(self.sigma_sq))
            for i, test_point in enumerate(test_data):
                log_prob[i] = c - 1./2 * np.sum((test_point - self.mu)**2) / self.sigma_sq        
        return log_prob

For class `classif_bayes`:

  - complete `classif_bayes.compute_predictions`

In [47]:
class classif_bayes:

    def __init__(self,models_ml, priors):
        self.models_ml = models_ml
        self.priors = priors
        if len(self.models_ml) != len(self.priors):
            print('The number of ML models must be equal to the number of priors!')
        self.n_classes = len(self.models_ml)
			
    # Returns a matrix of size nb. of test ex. times number of classes containing the log
    # probabilities of each test example under each model, trained by ML.	
    def compute_predictions(self, test_data):

        log_pred = np.empty((test_data.shape[0],self.n_classes))
        for i in range(self.n_classes):
            # Here we will have to use modeles_mv [i] and priors to fill in
            # each column of log_pred (it's more efficient to do a entire column at a time)
            log_pred[:,i] = np.transpose(model_ml[i].compute_predictions(test_data) * self.priors[i])
        return log_pred

Complete the code by calculating the maximum per class and displaying the decision area graph using the functions in `utilities.py`` 

In [51]:
# Here we provide an example where we do not divide the data into train and test set.
# it's up to you to do it like in demo 3
iris=np.loadtxt('iris.txt')
iris_train1=iris[0:50,:-1]
iris_train2=iris[50:100,:-1]
iris_train3=iris[100:150,:-1]

# We create a model per class (using maximum likelihood)
model_class1=gauss_ml(2)
model_class2=gauss_ml(2)
model_class3=gauss_ml(2)
model_class1.train(iris_train1)
model_class2.train(iris_train2)
model_class3.train(iris_train3)

# We create a list of all our models
# We do the same thing for the priors
# Here the priors are calculated exactly because we know the number of representatives per class. 
# Once you have created a train / test set, they need to be calculated exactly
model_ml=[model_class1,model_class2,model_class3]
priors=[0.3333,0.3333,0.3333]

# We create our classifier with our list of Gaussian models and our priors
classifier=classif_bayes(model_ml,priors)

# we can now calculate the log-probabilities according to our models
#log_prob=classifier.compute_predictions(iris_train3)

# it now remains to calculate the maximum per class for the classification
#max_prob = np.argmax(log_prob, axis = 1)
print(classifier.models_ml[0].n_dims)
utilities.gridplot(classifier, iris_train1[:,0:2], iris_train1[:,0:2])

2
[5.006 3.428 1.462 0.246]


ValueError: operands could not be broadcast together with shapes (2,) (4,) 

## Once you're done

- Change your code so that `gauss_ml` calculates a diagonal covariance matrix (where we estimate the variance for each component / trait of the input) or even a full covariance matrix.
- The `numpy.cov` and` numpy.var` commands will probably be useful.